SIN-393 - Introdução à Visão Computacional (2022-2)

# Aula 08 - Redes Neurais Convolucionais (Parte 1)

Prof. João Fernando Mari ([*joaofmari.github.io*](https://joaofmari.github.io/))

---

# Importando as bibliotecas necessárias
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

%matplotlib notebook

## Verificando o acesso à GPU
---

* Editar >> Configurações de notebook >> Acelerador de hardware
* Selecione GPU
* OK

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('\nDevice: {0}'.format(DEVICE))


Device: cuda


In [3]:
!nvidia-smi

Mon Nov 28 18:02:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 45%   27C    P8    N/A /  75W |      8MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Configurações para reprodutibilidade
---

In [4]:
np.random.seed(1234)

# Definimos alguns hiperparâmetros
---

In [5]:
# Nomes das classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Tamanho do lote (mini-batch)
batch_size = 8 # 4

# O conjunto de dados
---

In [6]:
# Define uma sequencia de transformações que serão aplicadas sobre as imagens dos datasets
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Datasets
# --------
# Conjunto de treinamento
dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
# Conjunto de testes
dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

# Dataloaders
# -----------
# Conjunto de treinamento
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                               shuffle=True, num_workers=2)
# Conjunto de testes
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,
                                              shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## Definindo uma Rede Neural Convolucional simples
---

In [7]:
class Net(nn.Module):
    """
    Input [3, 32, 32] 
    Conv1(3, 6, 5) [6, 28, 28] 
    Pool(2, 2) [6, 14, 14] 
    Conv2(6, 16, 5) [16, 10, 10]
    Pool(2, 2) [16, 5, 5]
    Flatten [400]
    Fc1 [120]
    Fc2 [84]
    Fc3 [10]
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
# Instância um objeto da classe Net
net = Net()

net = net.cuda() # Cuda

print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Definindo a função de perda e o otimizador
---

In [9]:
# Função de perda (loss) - Entrôpia cruzada
criterion = nn.CrossEntropyLoss()

# Otimizador - Stochastic Gradient Descent
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Treinando o modelo
---

In [10]:
# Itera ao longo do dataset por um número de épocas.
for epoch in range(2):  

    running_loss = 0.0

    # Treino
    # ------
    for i, (inputs, labels) in enumerate(dataloader_train, 0):
        
        inputs = inputs.to(DEVICE) 
        labels = labels.to(DEVICE) 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward 
        outputs = net(inputs)
        # Calcula a função de perda
        loss = criterion(outputs, labels)
        
        # backward
        loss.backward()
        
        # Otimiza os parâmetros (pesos)
        optimizer.step()

        # PRINTING...
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'Época {epoch + 1}: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('\nTreinamento finalizado!')

Época 1: 2.173
Época 1: 1.757
Época 1: 1.599
Época 2: 1.488
Época 2: 1.420
Época 2: 1.379

Treinamento finalizado!


## Testando o modelo
---

In [12]:
# Número de imagens classificadas corretamente
correct = 0
# Número total de imagens
total = 0

# Não é necessário calcular os gradientes.
with torch.no_grad():
    for inputs, labels in dataloader_test:
        
        inputs = inputs.to(DEVICE) 
        labels = labels.to(DEVICE) 
        
        # Calculo das saídas
        outputs = net(inputs)

        # A imagem é classificada de acordo com a maior saída
        _, predicted = torch.max(outputs.data, 1)

        # Atualiza o número de imagens
        total += labels.size(0)
        # Atualiza o número de classificações corretas
        correct += (predicted == labels).sum().item()

# Calcula a acurácia sobre o conjunto de testes
accuracy = 100 * correct // total

print(f'Acurácia da rede sobre o conjunto de testes: {accuracy} %')

Acurácia da rede sobre o conjunto de testes: 50 %


## Bibliografia
---
* PyTorch. Training a Classifier
    * https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
* Microsoft. Train your image classifier model with PyTorch.
    * https://learn.microsoft.com/en-us/windows/ai/windows-ml/tutorials/pytorch-train-model

